In [ ]:
%pip install librosa soundfile numpy scikit-learn pyaudio tensorflow

In [1]:
import librosa
import soundfile
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [2]:
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        result = np.array([])
        if chroma:
            stft = np.abs(librosa.stft(X))
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result


In [3]:
emotions = {
    '01':'neutral', '02':'calm', '03':'happy', '04':'sad',
    '05':'angry', '06':'fearful', '07':'disgust', '08':'surprised'
}
observed_emotions = ['calm', 'happy', 'fearful', 'disgust']

In [4]:
def load_data(test_size=0.25):
    x, y = [], []
    for file in glob.glob("D:\\Projects\\Machine Learning\\speech emotion detector\\data\\Actor_*\\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [5]:
x_train, x_test, y_train, y_test = load_data(test_size=0.25)

In [6]:
x_train.shape, x_test.shape

((576, 180), (192, 180))

In [7]:
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train))
y_test = to_categorical(le.transform(y_test))

In [8]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
model = Sequential()
model.add(Dense(256, input_shape=(x_train.shape[1],), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/100


18/18 [==============================] - 5s 94ms/step - loss: 1.4199 - accuracy: 0.3785 - val_loss: 1.1473 - val_accuracy: 0.4896
Epoch 2/100
18/18 [==============================] - 0s 21ms/step - loss: 1.0726 - accuracy: 0.5365 - val_loss: 1.0259 - val_accuracy: 0.5469
Epoch 3/100
18/18 [==============================] - 0s 28ms/step - loss: 0.9588 - accuracy: 0.6163 - val_loss: 0.9550 - val_accuracy: 0.5990
Epoch 4/100
18/18 [==============================] - 0s 23ms/step - loss: 0.8588 - accuracy: 0.6424 - val_loss: 0.9071 - val_accuracy: 0.6198
Epoch 5/100
18/18 [==============================] - 1s 34ms/step - loss: 0.7736 - accuracy: 0.6910 - val_loss: 0.8881 - val_accuracy: 0.6562
Epoch 6/100
18/18 [==============================] - 0s 20ms/step - loss: 0.6963 - accuracy: 0.7292 - val_loss: 0.8524 - val_accuracy: 0.6250
Epoch 7/100
18/18 [==============================] - 0s 26ms/step - loss: 0.6565 - accuracy: 0.7552 - val_loss: 0.8407 - val_accuracy: 0.6771
Epoc

In [12]:
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

6/6 [==============================] - 0s 6ms/step - loss: 2.1480 - accuracy: 0.7708
Test Accuracy: 77.08%


In [18]:
def predict_emotion(file_path):
    feature = extract_feature(file_path, mfcc=True, chroma=True, mel=True)
    feature = scaler.transform([feature])
    prediction = model.predict(feature)
    emotion = le.inverse_transform([np.argmax(prediction)])
    return emotion[0]

# Example
print(predict_emotion("D:\\Projects\\Machine Learning\\speech emotion detector\\test\\sad.wav"))

1/1 [==============================] - 0s 58ms/step
calm


In [ ]:
# Save model
model.save("emotion_model.h5")

# Save scaler and label encoder
import pickle
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)
